## Etapa 1: Instalação da bibliotecas


In [1]:
!pip uninstall -y tensorflow #Comando necessário, pois o TensorFlow-gpu não desinstala a versão mais recente do Tensorflow, pode gerar conflitos.

Found existing installation: tensorflow 2.7.0
Uninstalling tensorflow-2.7.0:
  Successfully uninstalled tensorflow-2.7.0


In [2]:
!pip install tensorflow==2.0.0 #Algumas bibliotecas ainda não tem compatibilidade com versões acima, favor utilizar somente essa versão.

     |████████████████████████████████| 86.3 MB 43 kB/s 
     |████████████████████████████████| 3.8 MB 45.0 MB/s 
     |████████████████████████████████| 50 kB 6.7 MB/s 
     |████████████████████████████████| 449 kB 47.8 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=9c217eb18098e9ce78c0d74a22333905bed5460e2b19f19369b908461acf45b8
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Success

In [3]:
!pip install -q tensorflow-gpu==2.0.0 #Algumas bibliotecas ainda não tem compatibilidade com versões acima, favor utilizar somente essa versão.

     |████████████████████████████████| 380.8 MB 41 kB/s 


In [4]:
!pip install pandas-datareader

## Etapa 2: Importação das bibliotecas

In [5]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader
from pandas.util.testing import assert_frame_equal #import alterado

from tqdm import tqdm_notebook, tqdm
from collections import deque

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  


In [6]:
tf.__version__

'2.0.0'

## Etapa 3: Construção da IA para negociação de ações

In [7]:
class AI_Trader():
  
  def __init__(self, state_size, action_space = 3, model_name = "AITrader"):
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen = 2000)
    self.model_name = model_name
    
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
    self.model = self.model_builder()
    
  def model_builder(self):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.Input(shape=(self.state_size,)))
    model.add(tf.keras.layers.Dense(units = 32, activation = "relu"))
    model.add(tf.keras.layers.Dense(units = 64, activation = "relu"))
    model.add(tf.keras.layers.Dense(units = 128, activation = "relu"))
    model.add(tf.keras.layers.Dense(units = self.action_space, activation = "linear"))
    model.compile(loss = "mse", optimizer = tf.keras.optimizers.Adam(lr = 0.001))
    return model

  
  def trade(self, state):
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    
    actions = self.model.predict(state)
    return np.argmax(actions[0])
  
  def batch_train(self, batch_size):
    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])
      
    for state, action, reward, next_state, done in batch:
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
        
      target = self.model.predict(state)
      target[0][action] = reward
      
      self.model.fit(state, target, epochs=1, verbose=0)
      
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay
      

In [ ]:
if random.random() <= self.epsilon:
  return random.randrange(self.action_space)

actions = self.model.predict(state)
return np.argmax(actions[0])

In [ ]:
random.randrange(self.action_space)

In [ ]:
a=8
a

8

## Etapa 4: Pré-processamento da base de dados

### Definição de funções auxiliares

#### Sigmoid

In [8]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [9]:
sigmoid(0.5)

0.6224593312018546

#### Formatação de preços

In [10]:
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

In [11]:
stocks_price_format(100)

'$ 100.000000'

#### Carregador da base de dados

In [12]:
!pip install -q yfinance

     |████████████████████████████████| 6.4 MB 7.5 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [13]:
import yfinance as yf

In [14]:
#dataset = data_reader.DataReader("AAPL", data_source = "yahoo")
dataset = yf.download("AAPL", start='2016-06-02')

[*********************100%***********************]  1 of 1 completed


In [15]:
dataset.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2016-06-02,24.400000,24.459999,24.157499,24.430000,22.754786,160766400
2016-06-03,24.447500,24.567499,24.362499,24.480000,22.801355,114019600
2016-06-06,24.497499,25.472500,24.387501,24.657499,22.966686,93170000
2016-06-07,24.812500,24.967501,24.740000,24.757500,23.059828,89638000
2016-06-08,24.754999,24.889999,24.670000,24.735001,23.038874,83392400


In [16]:
str(dataset.index[0]).split()[0]

'2016-06-02'

In [17]:
dataset.index[-1]

Timestamp('2022-01-11 00:00:00')

In [18]:
dataset['Close']

Date
2016-06-02     24.430000
2016-06-03     24.480000
2016-06-06     24.657499
2016-06-07     24.757500
2016-06-08     24.735001
                 ...    
2022-01-05    174.919998
2022-01-06    172.000000
2022-01-07    172.169998
2022-01-10    172.190002
2022-01-11    174.419998
Name: Close, Length: 1414, dtype: float64

In [19]:
def dataset_loader(stock_name):
  #dataset = data_reader.DataReader(stock_name, data_source = "yahoo")
  dataset = yf.download(stock_name, start='2016-06-02')
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  close = dataset['Close']
  return close

### Criador de estados

In [20]:
0 - 5 + 1

-4

In [21]:
20 - 5 + 1

16

In [22]:
dataset[16:21]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2016-06-24,23.227501,23.665001,23.162500,23.350000,21.748844,301245600
2016-06-27,23.250000,23.262501,22.875000,23.010000,21.432161,181958400
2016-06-28,23.225000,23.415001,23.035000,23.397499,21.793085,161779600
2016-06-29,23.492500,23.637501,23.407499,23.600000,21.981701,146124000
2016-06-30,23.610001,23.942499,23.575001,23.900000,22.261127,143345600


In [23]:
def state_creator(data, timestep, window_size):
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep + 1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep + 1])
    
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i + 1] - windowed_data[i]))
    
  return np.array([state]), windowed_data

### Carregando a base de dados

In [24]:
stock_name = "AAPL"
data = dataset_loader(stock_name)

[*********************100%***********************]  1 of 1 completed


In [25]:
s, w = state_creator(data, 0, 5)

In [26]:
s

array([[0.5, 0.5, 0.5, 0.5]])

In [27]:
w

[24.43000030517578,
 24.43000030517578,
 24.43000030517578,
 24.43000030517578,
 24.43000030517578]

## Etapa 5: Treinando a IA

### Configuração dos hyper parâmetros

In [28]:
window_size = 10
episodes = 1000
batch_size = 32
data_samples = len(data) - 1

In [29]:
data_samples

1413

### Definição do modelo

In [30]:
trader = AI_Trader(window_size)

In [31]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


### Loop de treinamento

In [ ]:
for episode in range(1, episodes + 1):
  print("Episode: {}/{}".format(episode, episodes))
  state = state_creator(data, 0, window_size + 1)
  total_profit = 0
  trader.inventory = []
  for t in tqdm(range(data_samples)):
    action = trader.trade(state)
    next_state = state_creator(data, t + 1, window_size + 1)
    reward = 0
    
    if action == 1: # Comprando uma ação
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stocks_price_format(data[t]))
    elif action == 2 and len(trader.inventory) > 0: # Vendendo uma ação  
      buy_price = trader.inventory.pop(0)
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price))
      
    if t == data_samples - 1:
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print("########################")
      print("Total profit: {}".format(total_profit))
      print("########################")
      
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
     
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))
    